In [1]:
import os
import numpy as np
from flask import Flask, request, jsonify, render_template, session
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Initialize the Flask app
app = Flask(__name__)
# app.secret_key = 'your_secret_key'  # For session management

# Load the pre-trained model
model_path = "Model/model_v1_inceptionV3.h5"  # Update the path to your model
food_recognition_model = load_model(model_path)

# Define the list of food classes (based on your dataset structure)
food_classes = [
    "burger", "butter_naan", "chai", "chapati", "chole_bhature",
    "dal_makhani", "dhokla", "fried_rice", "idli", "jalebi",
    "kaathi_rolls", "kadai_paneer", "kulfi", "masala_dosa", "momos",
    "paani_puri", "pav_bhaji", "pizza", "samosa"
]


# Define routes for your Flask app
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/image_upload')
def image_upload():
    return render_template('image_upload.html')

@app.route('/health')
def health():
    return render_template('health.html')

@app.route('/gender')
def gender():
    return render_template('gender.html')

@app.route('/upload', methods=['POST'])
def upload_image():
    file = request.files.get('image')  # Get the uploaded file
    if file:
        try:
            # Use the file's stream (convert to BytesIO if necessary)
            from io import BytesIO
            file_stream = BytesIO(file.read())  # Convert to BytesIO object
            
            # Load the image using Keras's load_img
            image = load_img(file_stream, target_size=(299, 299))  # InceptionV3 input size
            image_array = img_to_array(image) / 255.0  # Normalize pixel values to [0, 1]
            image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

            # Make predictions using the food recognition model
            predictions = food_recognition_model.predict(image_array)
            food_index = np.argmax(predictions)

            # Get the predicted food label
            if food_index < len(food_classes):
                food_label = food_classes[food_index]
            else:
                return jsonify({'error': 'Prediction index out of range'}), 400

            # Check for allergens in user health information (stored in session)
            user_allergies = session.get('allergies', [])
            detected_allergens = detect_allergens(food_label)

            # Compare detected allergens with user's allergies
            if any(allergen in user_allergies for allergen in detected_allergens):
                message = f"Warning! The food '{food_label}' contains allergens you are allergic to: {', '.join(detected_allergens)}."
            else:
                message = f"No allergens detected in '{food_label}'. It should be safe to consume."

            # Render the result page
            return render_template('result.html', food_label=food_label, message=message)
        except Exception as e:
            return jsonify({'error': str(e)}), 500

    return jsonify({'error': 'No file uploaded'}), 400



def detect_allergens(food_label):
    """
    Simulated function to detect allergens based on food label.
    Replace with your actual allergy detection logic if needed.
    """
    # Example mapping of allergens to food categories
    allergen_mapping = {
        'burger': ['Gluten'],
        'pizza': ['Dairy', 'Gluten'],
        'samosa': ['Gluten'],
        # Add more mappings as needed
    }
    return allergen_mapping.get(food_label, [])

@app.route('/submit_health_info', methods=['POST'])
def submit_health_info():
    data = request.json
    allergies = data.get('allergies', [])
    session['allergies'] = allergies  # Store allergies in the session
    return jsonify({'message': 'Health information received'}), 200

if __name__ == '__main__':
    app.run(debug=True)


c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


SystemExit: 1

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import pandas as pd

# Create data for food items, ingredients, and common allergies
data = {
    "Food Item": [
        "burger", "butter_naan", "chai", "chapati", "chole_bhature",
        "dal_makhani", "dhokla", "fried_rice", "idli", "jalebi",
        "kaathi_rolls", "kadai_paneer", "kulfi", "masala_dosa", "momos",
        "paani_puri", "pakode", "pav_bhaji", "pizza", "samosa"
    ],
    "Ingredients": [
        "Bread, patty (beef/chicken/veg), lettuce, tomato, cheese, mayo",
        "All-purpose flour, butter, salt, water",
        "Tea leaves, milk, water, sugar, spices (cardamom, ginger)",
        "Whole wheat flour, water, salt",
        "Chickpeas, flour, spices, oil, yogurt",
        "Lentils, butter, cream, spices (cumin, garam masala)",
        "Gram flour, yogurt, mustard seeds, green chilies",
        "Rice, vegetables, soy sauce, oil, spices",
        "Rice, lentils, water, salt",
        "Flour, sugar, ghee, cardamom",
        "Flatbread, spiced filling (veg/chicken), chutney",
        "Paneer, tomatoes, cream, spices",
        "Milk, sugar, cardamom, saffron",
        "Rice, lentils, potato filling, spices, ghee",
        "Flour, vegetables/meat, spices, soy sauce",
        "Flour, water, spiced water, chickpeas",
        "Gram flour, onions, spices, oil",
        "Pav (bread), mashed vegetables, butter, spices",
        "Flour, cheese, tomato sauce, toppings (veg/non-veg)",
        "Flour, potato filling, peas, spices, oil"
    ],
    "Common Allergies": [
        "Gluten, dairy, egg (in mayo or bread), soy (in sauces)",
        "Gluten, dairy",
        "Dairy, caffeine",
        "Gluten",
        "Gluten, legumes (chickpeas), dairy",
        "Legumes, dairy",
        "Legumes (gram flour), dairy",
        "Soy, gluten (in soy sauce), sesame (optional)",
        "Legumes (lentils)",
        "Gluten (flour), dairy (ghee)",
        "Gluten",
        "Dairy",
        "Dairy",
        "Legumes (lentils), dairy (ghee)",
        "Gluten, soy",
        "Gluten, legumes (chickpeas)",
        "Legumes (gram flour), gluten (cross-contamination)",
        "Gluten, dairy",
        "Gluten, dairy, soy",
        "Gluten, legumes (peas), dairy (optional)"
    ]
}

# Convert data to DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('Food_Allergy_Data.csv', index=False)

print("CSV file 'Food_Allergy_Data.csv' created successfully.")


CSV file 'Food_Allergy_Data.csv' created successfully.


In [2]:
# Assuming `train_generator` was used to train the model
# Save the mapping during training:
class_indices = train_generator.class_indices  # {'burger': 0, 'butter_naan': 1, ...}
# Reverse the mapping to align predictions with class labels
food_classes = {v: k for k, v in class_indices.items()}


NameError: name 'train_generator' is not defined

In [3]:
predictions = food_recognition_model.predict(image_array)
print("Predictions:", predictions)  # Debugging step
food_index = np.argmax(predictions)
print("Predicted Index:", food_index)


NameError: name 'image_array' is not defined